# Fantasy Premier League team optimisation
### Intro

This is an attempt to optimise my fantasy football squad selection, picking an initial team and subsequent transfers. I've used data from https://github.com/vaastav/Fantasy-Premier-League/ which includes an 'xP' field, which isn't exactly what's required but is a good enough placeholder until the season starts with more accurate player predictions.

Player points prediction is probably the most technical element of a task like this. It would involve not only gathering the most common data but also searching for alternative data that could improve accuracy, such as betting odds for various markets or even something as stupid as the tweets of the player may improve accuracy. There are a number of available models currently out there, put together by people who have thought a lot more about the problem than I have, so initially this project is purely an optimisation problem based on the predicted points and any other factors useful for creating a team.

The optimisation is carried out using `pulp`. I'd considered converting the problem into `pytorch` to crunch the numbers on a GPU, but it's only taking 5 seconds or so to run over a CPU for 6 gameweeks which is fast enough for now.

First grabbing the data:
```
git clone https://github.com/vaastav/Fantasy-Premier-League.git
```

### Data prep

In [1]:
import pulp
import pandas as pd
import chardet
import os
import seaborn as sns
import matplotlib.pyplot as plt
import requests

Load data from fpl api

In [2]:
r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
fpl_data = r.json()
element_data = pd.DataFrame(fpl_data['elements'])

In [3]:
element_data.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,133,82,762,324,780,336,711,296,0.00,0.00
1,True,True,0.0,0.0,205651,0,0,-5,5,2,...,44,20,782,80,206,26,204,31,0.90,0.30
2,True,True,0.0,0.0,226597,0,0,3,-3,2,...,55,4,159,51,40,3,15,6,1.07,0.38
3,True,True,0.0,0.0,219847,0,0,-3,3,1,...,13,4,791,86,24,9,66,12,1.03,0.34
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,665,56,465,56,603,67,629,70,0.00,0.00


In [4]:
type_data = pd.DataFrame(fpl_data['element_types']).set_index(['id'])
type_data.head()

,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
id,,,,,,,,,,,,
1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],82
2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],262
3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],341
4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],86
5,Managers,MNG,Manager,MNG,0,None,None,0,0,True,[],20


In [5]:
import time
import json

# The 'elements' key contains the list of players
players = fpl_data.get('elements', [])

# Dictionary to hold gameweek points per player by gameweek
player_gameweek_points = {}

# Step 2: For each player, fetch gameweek history from the element-summary endpoint
for player in players:
    player_id = player['id']
    player_name = player['web_name']
    summary_url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
    
    summary_response = requests.get(summary_url)
    if summary_response.status_code != 200:
        print(f"Error fetching data for player {player_name} (ID: {player_id}).")
        continue
    
    summary_data = summary_response.json()
    history = summary_data.get('history', [])
    
    # Create a dictionary mapping each gameweek ('round') to 'total_points'
    gameweek_points = {entry['round']: entry['total_points'] for entry in history}
    player_gameweek_points[player_name] = gameweek_points
    
    # Optional: pause to avoid overwhelming the server (be respectful)
    time.sleep(0.1)


In [6]:
import pandas as pd

# Convert dictionary to DataFrame
df = pd.DataFrame.from_dict(player_gameweek_points, orient='index')

# Optionally, rename columns (e.g., to "GW1", "GW2", etc.)
df.columns = [f"GW{col}" for col in df.columns]
df.index.name = 'name'
df = df.fillna(0)

df = df.reset_index()  # now 'name' is a normal column

# Unpivot the wide table to long format
df_actual = pd.melt(
    df,
    id_vars="name",          # columns to keep fixed
    var_name="gameweek",     # new column name for former column headers
    value_name="points"      # new column name for values
)

print(df_actual.head())

           name gameweek  points
0  Fábio Vieira      GW1     0.0
1       G.Jesus      GW1     0.0
2       Gabriel      GW1     6.0
3       Havertz      GW1    12.0
4          Hein      GW1     0.0


### Optimisation
#### Single gameweek optimisation

First we'll just build an optimisation model for a single gameweek

In [8]:
df = pd.read_csv('../scr/preds.csv')
df = df.melt(id_vars=["name", "team", "position", "value"], 
                    value_vars=["32", "33", "34", "35", "36"],
                    var_name="gameweek", 
                    value_name="xP")
df['season'] = "2024-25"

df.head()

,name,team,position,value,gameweek,xP,season
0,M.Salah,LIV,MID,13.8,32,8.3,2024-25
1,Palmer,CHE,MID,10.7,32,8.2,2024-25
2,Isak,NEW,FWD,9.5,32,10.4,2024-25
3,Saka,ARS,MID,10.3,32,6.1,2024-25
4,B.Fernandes,MUN,MID,8.6,32,4.2,2024-25


In [9]:
cols_opt = [
    "name",
    "position",
    "team",
    "xP",
    "value",
    "gameweek",
    "season"
    ]

df_optimisation = df[cols_opt].sort_values(by=['gameweek', 'season'], ascending=[True, True])
df_optimisation['gameweek'] = df_optimisation['gameweek'].astype('int')
df_optimisation['value'] = df_optimisation['value'].astype('float64')
df_optimisation['xP'] = df_optimisation['xP'].astype('float64')
print(df_optimisation)

                 name position team    xP  value  gameweek   season
0             M.Salah      MID  LIV   8.3   13.8        32  2024-25
1              Palmer      MID  CHE   8.2   10.7        32  2024-25
2                Isak      FWD  NEW  10.4    9.5        32  2024-25
3                Saka      MID  ARS   6.1   10.3        32  2024-25
4         B.Fernandes      MID  MUN   4.2    8.6        32  2024-25
...               ...      ...  ...   ...    ...       ...      ...
1430          Odobert      MID  TOT   1.2    5.3        36  2024-25
1431           Botman      DEF  NEW   1.4    4.4        36  2024-25
1432  De Cordova-Reid      MID  LEI   1.1    5.2        36  2024-25
1433             Gray      MID  TOT   1.1    4.6        36  2024-25
1434       Jota Silva      MID  NFO   1.2    5.9        36  2024-25

[1435 rows x 7 columns]


In [10]:
def solve_single_period_fpl(budget, data, gameweek, season):
    # Filter data for the specified gameweek and season
    filtered_data = data[(data['gameweek'] == gameweek) & (data['season'] == season)]
    
    # Create the model
    model = pulp.LpProblem("FPL_Optimization", pulp.LpMaximize)

    # Variables
    squad = pulp.LpVariable.dicts("squad", filtered_data.index, cat='Binary')
    lineup = pulp.LpVariable.dicts("lineup", filtered_data.index, cat='Binary')
    captain = pulp.LpVariable.dicts("captain", filtered_data.index, cat='Binary')
    vicecap = pulp.LpVariable.dicts("vicecap", filtered_data.index, cat='Binary')

    # Objective
    model += pulp.lpSum(filtered_data.loc[p, 'xP'] * (lineup[p] + captain[p] + 0.1 * vicecap[p]) for p in filtered_data.index)

    # Constraints
    # Squad size
    model += pulp.lpSum(squad[p] for p in filtered_data.index) == 15

    # Lineup size
    model += pulp.lpSum(lineup[p] for p in filtered_data.index) == 11

    # Captain and vice-captain
    model += pulp.lpSum(captain[p] for p in filtered_data.index) == 1
    model += pulp.lpSum(vicecap[p] for p in filtered_data.index) == 1

    # Lineup, captain, and vice-captain must be in squad
    for p in filtered_data.index:
        model += lineup[p] <= squad[p]
        model += captain[p] <= lineup[p]
        model += vicecap[p] <= lineup[p]
        model += captain[p] + vicecap[p] <= 1

    # Position constraints
    for pos in filtered_data['position'].unique():
        model += pulp.lpSum(squad[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) == {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[pos]
        if pos == 'GK':
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) == 1
        elif pos in ['DEF', 'MID']:
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) >= 3
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) <= 5
        else:  # FWD
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) >= 1
            model += pulp.lpSum(lineup[p] for p in filtered_data.index if filtered_data.loc[p, 'position'] == pos) <= 3

    # Budget constraint (player prices are 10x)
    model += pulp.lpSum(filtered_data.loc[p, 'value'] * squad[p] for p in filtered_data.index) <= budget

    # Team limit
    for team in filtered_data['team'].unique():
        model += pulp.lpSum(squad[p] for p in filtered_data.index if filtered_data.loc[p, 'team'] == team) <= 3

    # Solve the model
    model.solve()

    # Process results
    if pulp.LpStatus[model.status] == 'Optimal':
        picks = []
        for p in filtered_data.index:
            if squad[p].value() > 0.5:
                player_data = filtered_data.loc[p]
                is_captain = 1 if captain[p].value() > 0.5 else 0
                is_lineup = 1 if lineup[p].value() > 0.5 else 0
                is_vice = 1 if vicecap[p].value() > 0.5 else 0
                picks.append([
                    player_data['name'],
                    player_data['position'],
                    player_data['team'],
                    player_data['value'],
                    player_data['xP'],
                    is_lineup,
                    is_captain,
                    is_vice
                ])

        picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain'])
        picks_df = picks_df.sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, False])
        
        total_xp = pulp.value(model.objective)
        print(f'Total expected points for budget {budget}: {total_xp}')

        return {'model': model, 'picks': picks_df, 'total_xp': total_xp}
    else:
        print(f"Optimization failed. Status: {pulp.LpStatus[model.status]}")
        return None


In [12]:
results = solve_single_period_fpl(103.1, df_optimisation, 32, '2024-25')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/49aa8fc87dee48d7b18e50d3f8143837-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/49aa8fc87dee48d7b18e50d3f8143837-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1189 COLUMNS
At line 9197 RHS
At line 10382 BOUNDS
At line 11531 ENDATA
Problem MODEL has 1184 rows, 1148 columns and 4859 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 86.3575 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 861 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 subst

In [13]:
if results is not None:
    picks = results['picks']
    print(picks)
else:
    print("No valid solution found.")
    
# Get total expected points of the squad
total_xp = round(picks['xP'].sum(),2)

# Get total cost of the squad
total_cost = picks['price'].sum() / 10

         name  pos team  price    xP  lineup  captain  vicecaptain
9       Schär  DEF  NEW    5.4   6.5       1        0            0
7    Gvardiol  DEF  MCI    6.1   4.2       1        0            0
11  Cucurella  DEF  CHE    5.3   4.1       1        0            0
2        Isak  FWD  NEW    9.5  10.4       1        1            0
4      Mateta  FWD  CRY    7.6   7.0       1        0            0
8   Henderson   GK  CRY    4.6   5.7       1        0            0
0     M.Salah  MID  LIV   13.8   8.3       1        0            1
1      Palmer  MID  CHE   10.7   8.2       1        0            0
6      Gordon  MID  NEW    7.4   7.9       1        0            0
5      I.Sarr  MID  CRY    5.7   6.7       1        0            0
3        Saka  MID  ARS   10.3   6.1       1        0            0
14    Quansah  DEF  LIV    3.9   1.5       0        0            0
13    Burgess  DEF  IPS    3.8   1.0       0        0            0
12       Daka  FWD  LEI    4.8   2.7       0        0         

#### Multi-period optimisation
Now for the trickier task of optimising over multiple gameweeks, initially just assuming a wildcard (ie no current team) over n gameweeks.

In [33]:
def solve_multi_period_fpl(budget, data, start_gameweek, end_gameweek, season, bench_boost_gameweek, free_hit_gameweek):

    # Validate bench_boost_gameweek: 0 means no bench boost.
    if bench_boost_gameweek != 0 and (bench_boost_gameweek < start_gameweek -1 or bench_boost_gameweek > end_gameweek +1):
        raise ValueError("bench_boost_gameweek must be 0 or between start_gameweek and end_gameweek")
    # Validate free_hit_gameweek: 0 means no free hit; also, free hit should not be in the first gameweek.
    if free_hit_gameweek != 0:
        if free_hit_gameweek < start_gameweek or free_hit_gameweek > end_gameweek:
            raise ValueError("free_hit_gameweek must be 0 or between start_gameweek and end_gameweek")
        if free_hit_gameweek == start_gameweek:
            raise ValueError("free_hit_gameweek cannot be the first gameweek")
    
    # Filter data for the specified gameweeks and season.
    filtered_data = data[(data['gameweek'] >= start_gameweek) & 
                         (data['gameweek'] <= end_gameweek) & 
                         (data['season'] == season)]
    
    # Create a dictionary for quick access to player data.
    player_data = {(row['name'], row['gameweek']): row for _, row in filtered_data.iterrows()}
    
    # Create the optimization model.
    model = pulp.LpProblem("Multi_Period_FPL_Optimization", pulp.LpMaximize)

    # Sets.
    gameweeks = range(start_gameweek, end_gameweek + 1)
    players = filtered_data['name'].unique()
    positions = filtered_data['position'].unique()
    teams = filtered_data['team'].unique()

    # Decision Variables.
    squad = pulp.LpVariable.dicts("squad", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    lineup = pulp.LpVariable.dicts("lineup", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    captain = pulp.LpVariable.dicts("captain", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    vicecap = pulp.LpVariable.dicts("vicecap", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_in = pulp.LpVariable.dicts("transfer_in", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_out = pulp.LpVariable.dicts("transfer_out", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    free_transfers = pulp.LpVariable.dicts("free_transfers", gameweeks, lowBound=0, upBound=5, cat='Integer')
    paid_transfers = pulp.LpVariable.dicts("paid_transfers", gameweeks, lowBound=0, cat='Integer')

    # Objective:
    # - In the bench boost week (if > 0) use the entire squad (15 players), otherwise just the starting lineup.
    # - Transfer costs are applied in all weeks except the free hit week.
    model += pulp.lpSum(
        player_data.get((p, gw), {}).get('xP', 0) * (
            (squad[p, gw] if bench_boost_gameweek > 0 and gw == bench_boost_gameweek else lineup[p, gw])
            + captain[p, gw] + 0.1 * vicecap[p, gw]
        )
        for p in players for gw in gameweeks
    ) - pulp.lpSum(
        4 * paid_transfers[gw] for gw in gameweeks if gw != free_hit_gameweek
    )
    
    # Starting free transfers (typically 0 for the first gameweek).
    model += free_transfers[start_gameweek] == 0

    # Constraints that apply in every gameweek.
    for gw in gameweeks:
        # Squad size.
        model += pulp.lpSum(squad[p, gw] for p in players) == 15

        # Starting lineup size.
        model += pulp.lpSum(lineup[p, gw] for p in players) == 11

        # Exactly one captain and one vice-captain.
        model += pulp.lpSum(captain[p, gw] for p in players) == 1
        model += pulp.lpSum(vicecap[p, gw] for p in players) == 1

        # Lineup, captain, and vice-captain must be members of the squad.
        for p in players:
            model += lineup[p, gw] <= squad[p, gw]
            model += captain[p, gw] <= lineup[p, gw]
            model += vicecap[p, gw] <= lineup[p, gw]
            model += captain[p, gw] + vicecap[p, gw] <= 1

        # Position constraints.
        for pos in positions:
            # Ensure the correct number of players per position in the squad.
            model += pulp.lpSum(squad[p, gw] for p in players 
                                if player_data.get((p, gw), {}).get('position') == pos) \
                     == {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[pos]
            # For the starting lineup, enforce positional minimums/maximums.
            if pos == 'GK':
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) == 1
            elif pos in ['DEF', 'MID']:
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) >= 3
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) <= 5
            else:  # FWD.
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) >= 1
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) <= 3

        # Budget constraint.
        model += pulp.lpSum(player_data.get((p, gw), {}).get('value', 0) * squad[p, gw] for p in players) <= budget

        # Team limit (max 3 players from the same team).
        for team in teams:
            model += pulp.lpSum(squad[p, gw] for p in players 
                                if player_data.get((p, gw), {}).get('team') == team) <= 3

        # Transfer constraints.
        # Handle free hit weeks specially.
        if gw > start_gameweek:
            # ----- Free Hit Week -----
            if free_hit_gameweek != 0 and gw == free_hit_gameweek:
                # In the free hit week, allow a completely new squad.
                # Do not enforce the continuity constraint.
                model += free_transfers[gw] == free_transfers[gw-1]
                # (Transfers in/free hit week are not counted, so we do not add transfer_in/out constraints.)
            
            # ----- Week After Free Hit: Reversion -----
            elif free_hit_gameweek != 0 and gw == free_hit_gameweek + 1:
                # Force the squad to revert to the team from before the free hit.
                for p in players:
                    model += squad[p, gw] == squad[p, free_hit_gameweek - 1]
                # Also reset free transfers as if no transfers had been made in the free hit week.
                model += free_transfers[gw] == free_transfers[free_hit_gameweek - 1] + 1
            
            # ----- Normal Weeks -----
            else:
                model += free_transfers[gw] == free_transfers[gw-1] + 1 - pulp.lpSum(transfer_in[p, gw-1] for p in players)
                model += free_transfers[gw] <= 5 
                model += pulp.lpSum(transfer_in[p, gw] for p in players) == pulp.lpSum(transfer_out[p, gw] for p in players)
                model += pulp.lpSum(transfer_in[p, gw] for p in players) == free_transfers[gw] + paid_transfers[gw]
                for p in players:
                    model += squad[p, gw] == squad[p, gw-1] + transfer_in[p, gw] - transfer_out[p, gw]
                    model += transfer_in[p, gw] + transfer_out[p, gw] <= 1

    # Solve the model.
    model.solve()

    # Process and return results.
    if pulp.LpStatus[model.status] == 'Optimal':
        results = []
        for gw in gameweeks:
            picks = []
            transfers_in = []
            transfers_out = []
            for p in players:
                if (squad[p, gw].value() or 0) > 0.5:
                    player_info = player_data.get((p, gw), {})
                    is_captain = 1 if (captain[p, gw].value() or 0) > 0.5 else 0
                    is_lineup = 1 if (lineup[p, gw].value() or 0) > 0.5 else 0
                    is_vice = 1 if (vicecap[p, gw].value() or 0) > 0.5 else 0
                    picks.append([
                        p,
                        player_info.get('position', ''),
                        player_info.get('team', ''),
                        player_info.get('value', 0),
                        player_info.get('xP', 0),
                        is_lineup,
                        is_captain,
                        is_vice
                    ])
                if gw > start_gameweek:
                    if (transfer_in[p, gw].value() or 0) > 0.5:
                        transfers_in.append(p)
                    if (transfer_out[p, gw].value() or 0) > 0.5:
                        transfers_out.append(p)

            picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain'])
            picks_df = picks_df.sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, False])
            
            results.append({
                'gameweek': gw,
                'picks': picks_df,
                'transfers_in': transfers_in,
                'transfers_out': transfers_out,
                'free_transfers': free_transfers[gw].value(),
                'paid_transfers': paid_transfers[gw].value()
            })

        total_xp = pulp.value(model.objective)
        print(f'Total expected points for all gameweeks: {total_xp}')

        return {'results': results, 'total_xp': total_xp}
    else:
        print(f"Optimization failed. Status: {pulp.LpStatus[model.status]}")
        return None

In [38]:
result = solve_multi_period_fpl(102.3, df_optimisation, 32, 36, '2024-25', bench_boost_gameweek = 33, free_hit_gameweek = 34)

if result:
    for gw_result in result['results']:
        print(f"Gameweek {gw_result['gameweek']}:")
        print(gw_result['picks'])
        if gw_result['gameweek'] > 1:
            print("Transfers In:", gw_result['transfers_in'])
            print("Transfers Out:", gw_result['transfers_out'])
        print(f"Free Transfers: {gw_result['free_transfers']}")
        print(f"Paid Transfers: {gw_result['paid_transfers']}")
        print("\n")

    print(f"Total Expected Points: {result['total_xp']}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/18470d6a5783458b8b8bb7f1b2acca1e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/ss/kjxldrfs36qgygzmg25sqy100000gn/T/18470d6a5783458b8b8bb7f1b2acca1e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7296 COLUMNS
At line 56461 RHS
At line 63753 BOUNDS
At line 71147 ENDATA
Problem MODEL has 7291 rows, 7393 columns and 30303 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 371.608 - 0.02 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 4260 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 80 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 80 strengthened rows, 0 su

In [39]:
if result:
    markdown_output = ""

    # Add picks for each gameweek to Markdown
    for gw_result in result['results']:
        markdown_output += f"### Gameweek {gw_result['gameweek']} Picks\n\n"
        markdown_output += gw_result['picks'].to_markdown(index=False)
        markdown_output += "\n\n"

    # Add transfers summary to Markdown
    markdown_output += "### Transfers Summary\n\n"

    transfer_records = []
    for gw_result in result['results'][1:]:
        transfers_in = gw_result['transfers_in']
        transfers_out = gw_result['transfers_out']
        max_transfers = max(len(transfers_in), len(transfers_out))

        for i in range(max_transfers):
            transfer_in = transfers_in[i] if i < len(transfers_in) else ''
            transfer_out = transfers_out[i] if i < len(transfers_out) else ''
            transfer_records.append({'Gameweek': gw_result['gameweek'], 'Transfers In': transfer_in, 'Transfers Out': transfer_out})

    # Create a single DataFrame for all transfers
    transfers_df = pd.DataFrame(transfer_records)

    # Convert transfers DataFrame to Markdown
    markdown_output += transfers_df.to_markdown(index=False)

    # Print the Markdown output
    print(markdown_output)
else:
    print("Optimization did not find an optimal solution.")

### Gameweek 32 Picks

| name       | pos   | team   |   price |   xP |   lineup |   captain |   vicecaptain |
|:-----------|:------|:-------|--------:|-----:|---------:|----------:|--------------:|
| Burn       | DEF   | NEW    |     4.4 |  5.9 |        1 |         0 |             0 |
| Gvardiol   | DEF   | MCI    |     6.1 |  4.2 |        1 |         0 |             0 |
| Saliba     | DEF   | ARS    |     6.3 |  4.2 |        1 |         0 |             0 |
| Isak       | FWD   | NEW    |     9.5 | 10.4 |        1 |         1 |             0 |
| Joao Pedro | FWD   | BRI    |     5.6 |  5.3 |        1 |         0 |             0 |
| Marmoush   | FWD   | MCI    |     7.5 |  4.8 |        1 |         0 |             0 |
| Henderson  | GK    | CRY    |     4.6 |  5.7 |        1 |         0 |             0 |
| M.Salah    | MID   | LIV    |    13.8 |  8.3 |        1 |         0 |             1 |
| Eze        | MID   | CRY    |     6.8 |  6.9 |        1 |         0 |             0 |
| I.Sarr 

In [23]:
df_pred = pd.DataFrame()

for gw_data in result["results"]:
    gw = gw_data["gameweek"]
    gw_picks = gw_data["picks"]
    gw_picks['gameweek'] = f"GW{gw}"
    df_pred = pd.concat([df_pred, gw_picks])

print(df_pred)

          name  pos team  price    xP  lineup  captain  vicecaptain gameweek
10       Schär  DEF  NEW    5.4   6.5       1        0            0     GW32
7     Gvardiol  DEF  MCI    6.1   4.2       1        0            0     GW32
8       Saliba  DEF  ARS    6.3   4.2       1        0            0     GW32
1         Isak  FWD  NEW    9.5  10.4       1        1            0     GW32
6   Joao Pedro  FWD  BRI    5.6   5.3       1        0            0     GW32
3     Marmoush  FWD  MCI    7.5   4.8       1        0            0     GW32
9    Henderson   GK  CRY    4.6   5.7       1        0            0     GW32
0      M.Salah  MID  LIV   13.8   8.3       1        0            1     GW32
4          Eze  MID  CRY    6.8   6.9       1        0            0     GW32
5       I.Sarr  MID  CRY    5.7   6.7       1        0            0     GW32
2         Saka  MID  ARS   10.3   6.1       1        0            0     GW32
13    J.Timber  DEF  ARS    5.6   3.6       0        0            0     GW32

In [43]:
df_optimisation.groupby('gameweek')['xP'].apply(lambda x: x.nlargest(50).sum())

gameweek
32    262.4
33    305.9
34    203.2
35    200.4
36    202.0
Name: xP, dtype: float64